# Traitement et stockage des données issues du scraping

Vous avez scrapé les données du site de livres et les avez stockées dans un fichier. 

L'objectif de ce notebook est de créer une base de données pour y stocker ces données.

In [205]:
import sqlite3
import pandas as pd

Lire les données du fichier sauvegardé en utilisant pandas.

In [206]:
# Lire les données du fichier que vous venez d'enregistrer
df_books = pd.read_csv("books_data.csv")
print(df_books.head())

                                   title   price rating availability
0                   A Light in the Attic  £51.77  Three     In stock
1                     Tipping the Velvet  £53.74    One     In stock
2                             Soumission  £50.10    One     In stock
3                          Sharp Objects  £47.82   Four     In stock
4  Sapiens: A Brief History of Humankind  £54.23   Five     In stock


## 1. Prétraitement des données

On souhaite créer la table _book_ contenant les attributs suivants : 
- id : INT, PK,
- title : TEXT,
- price : DECIMAL
- availability : BOOLEAN
- rating : INT [0:5]

Vérifier les types des colonnes du dataframe.

In [207]:
# Vérification des types de données
print(df_books.dtypes)


title           object
price           object
rating          object
availability    object
dtype: object


Dans les cellules qui suivent, des méthodes de traitement de données sont suggérées pour donner un aperçu de ce qu'il est possible de faire avec pandas.

**Il est tout à fait possible de faire autrement.**

Utiliser la méthode pandas [_astype_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) pour convertir la colonne de titre en chaîne de caractère.

In [208]:
# # Conversion de title en chaîne de caractères
# df_books["title"].astype(str)

# # Vérification du type de la colonne title
# print(df_books["title"].dtype)

Pour convertir la colonne de prix en nombre décimal, il est nécessaire d'utiliser une étape intermédiaire pour retirer le caractère "£".

Il est possible par exemple d'utiliser l'attribut [.str](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html) de la série "price".

In [209]:
# # Convertir la colonne price en type décimal


# # Supprimer le symbole £ et convertir en float

# #df_books["price"] = df_books["price"].str.replace("£", "").astype(float)

# df_books["price"] = df_books["price"].str.replace("£", "").astype(float)



# print(df_books["price"].dtype)

Convertir la colonne `availability` en boolen (True/False).

Quelles sont les valeurs possibles pour la colonne availability ?

In [ ]:
# # Valeurs possibles de la colonne availability
 #df_books["availability"].unique()   
# # Convertir la colonne availability en booléen
#df_books["availability"] = df_books["availability"].astype(bool)
#print(df_books["availability"].dtype)




0      True
1      True
2      True
3      True
4      True
       ... 
995    True
996    True
997    True
998    True
999    True
Name: availability, Length: 1000, dtype: bool


Créer une fonction qui prend en entrée la valeur de `availability` et qui renvoie True ou False en fonction de la valeur d'entrée.

In [211]:
#Fonction pour convertir la valeur de availability en booléen
def convert_availability(value : str) -> bool:
    """Convert the availability value to a boolean.

    Args:
        value (str): The availability status of the book.

    Returns:
        bool: True if the book is available, False otherwise.
    """

    if value == "In stock" :
        return True
    return False

    # Vérification des types de données après conversion
print(df_books.dtypes)

title           object
price           object
rating          object
availability    object
dtype: object


Utiliser la méthode [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) pour appliquer la fonction à la colonne `availability`.

In [212]:
# # Convertir la colonne availability en booléen (True/False)

# df_books["availability"] = df_books["availability"].apply(convert_availability)
# #df_books["availability"] = df_books["availability"].apply(bool)



# # Vérification du type de la colonne availability
# print(df_books["availability"].dtype)

Convertir la colonne _rating_ en chiffre en utilisant un dictionnaire `rating_map` et la méthode [_map_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html).

In [213]:
# #Dictionnaire associant les notes au format initial et les valeurs numérique
# ratings_map = {
#     "one": 1,
#     "two": 2,
#     "three": 3,
#     "four": 4,  
#     "five": 5    
# }

# df_books["rating"] = df_books["rating"].map(ratings_map)
# df_books["rating"] = df_books["rating"].astype(float) # Convertir en float

# # Vérification du type de la colonne rating
# print(df_books["rating"].dtype)

In [225]:

# Fonction pour convertir la disponibilité en booléen
def convert_availability(availability: str) -> bool:
    return "In stock" in availability

# Créer une fonction convert_types qui combine les traitements faits dans les cellules précédentes
def convert_types(df_books: pd.DataFrame) -> pd.DataFrame:
    
    """
    Convert the types of the DataFrame columns to appropriate types.

    Args:
        df_books (pd.DataFrame): The DataFrame containing book data.

    Returns:
        pd.DataFrame: The DataFrame with converted types.
    """
    
    df_books["title"] = df_books["title"].astype(str)
    df_books["price"] = df_books["price"].str.replace("£", "").astype(float)
    df_books["availability"] = df_books["availability"].apply(convert_availability).astype(bool)
    df_books["rating"] = df_books["rating"].map(ratings_map).astype(float)
    
    ratings_map = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}


    return df_books
print(df_books.dtypes)
print(df_books.head())



title           object
price           object
rating          object
availability      bool
dtype: object
                                   title   price rating  availability
0                   A Light in the Attic  £51.77  Three          True
1                     Tipping the Velvet  £53.74    One          True
2                             Soumission  £50.10    One          True
3                          Sharp Objects  £47.82   Four          True
4  Sapiens: A Brief History of Humankind  £54.23   Five          True


---
## 2. Insertion des données en base

Dans cette section :
- on créé une BDD sqlite  `book_store.db` (ou on se connecte à la base si elle existe déjà) en utilisant la bibliothèque python sqlite3,
- on insère les données prétraitées dans la BDD

Utiliser le [tutoriel](https://www.ionos.fr/digitalguide/sites-internet/developpement-web/sqlite3-avec-python/) pour l'utilisation de sqlite3.

Utiliser la fonction pandas adaptée qui permet d'insérer un dataframe dans une BDD.

In [217]:
# Création de la BDD et insertion des données

import sqlite3
# Connexion à la base de données SQLite (ou création si elle n'existe pas)
connection = sqlite3.connect("books_database.db")

print("Connected to the database successfully.")
# Création d'un curseur pour exécuter des requêtes SQL
df_books.to_sql('title',connection)








Connected to the database successfully.


1000

Vérifier le nombre de livres présents dans la BDD en utilisant sqlite3 et la requête SQL adaptée.

In [226]:
# Compter le nombre de livre dans la BDD
cursor = connection.cursor()

cursor.execute("SELECT COUNT(*) FROM book")
book_count = cursor.fetchone()[0]

print(f"Nombre de livres dans la base de données : {book_count}")



Nombre de livres dans la base de données : 1000
